## 1. Data Wrangling##

- We will source all the data required for this project
- Clean the data
    1. Impute missing values.
    2. Extract/Deduce interpretable features from existing columns.
    3. Standardize the datat types.
    4. Omit columns that are not usuable.

Index - https://caaspp-elpac.cde.ca.gov/caaspp/research_fixfileformatCAST19
Understanding Scores - https://caaspp-elpac.cde.ca.gov/caaspp/UnderstandingCASTResults

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
na_values = ['n/a', 'na', '-', '*']

df_all  = pd.read_csv("../data/cast_ca2019_all_csv_v1.txt", parse_dates=True, na_values=na_values)
df_entities = pd.read_csv("../data/cast_ca2019entities_csv.txt", encoding='latin1', parse_dates=True, na_values=na_values)


In [4]:
print(df_all.shape)
df_all.head()


(986765, 29)


,County Code,District Code,School Code,Filler,Test Year,Demographic ID,Test Type,Total Number Tested at Entity Level and Demographic,Total Number Tested at this Demographic with Valid Scores,Grade,...,Total Number of Students with Valid Scores,Life Sciences Domain Percent Below Standard,Life Sciences Domain Percent Near Standard,Life Sciences Domain Percent Above Standard,Physical Sciences Domain Percent Below Standard,Physical Sciences Domain Percent Near Standard,Physical Sciences Domain Percent Above Standard,Earth and Space Sciences Domain Percent Below Standard,Earth and Space Sciences Domain Percent Near Standard,Earth and Space Sciences Domain Percent Above Standard
0,0,0,0,NaN,2019,1,X,1477006.0,1474994.0,5,...,456221.0,45.09,42.59,12.32,44.12,41.58,14.30,41.61,46.71,11.68
1,0,0,0,NaN,2019,3,X,752970.0,751862.0,5,...,234372.0,45.64,41.96,12.40,45.70,39.76,14.53,43.15,45.53,11.33
2,0,0,0,NaN,2019,4,X,724036.0,723132.0,5,...,221849.0,44.51,43.26,12.23,42.45,43.49,14.06,39.98,47.97,12.05
3,0,0,0,NaN,2019,6,X,1285057.0,1283476.0,5,...,368072.0,36.81,48.15,15.04,35.78,46.74,17.48,33.50,52.22,14.28
4,0,0,0,NaN,2019,7,X,67405.0,67369.0,5,...,17976.0,20.08,51.35,28.57,18.92,46.77,34.31,17.49,54.03,28.48


In [5]:
print(df_all['Test Type'].value_counts())
print(df_all['Test Year'].value_counts())
print(df_all['Test ID'].value_counts())
#Since these are same throughout, they can be deleted

X    986765
Name: Test Type, dtype: int64
2019    986765
Name: Test Year, dtype: int64
17    986765
Name: Test ID, dtype: int64


In [6]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986765 entries, 0 to 986764
Data columns (total 29 columns):
County Code                                                  986765 non-null int64
District Code                                                986765 non-null int64
School Code                                                  986765 non-null int64
Filler                                                       0 non-null float64
Test Year                                                    986765 non-null int64
Demographic ID                                               986765 non-null int64
Test Type                                                    986765 non-null object
Total Number Tested at Entity Level and Demographic          751470 non-null float64
Total Number Tested at this Demographic with Valid Scores    751470 non-null float64
Grade                                                        986765 non-null int64
Test ID                                                  

Since 'Test ID' is 17 which is 'Science' , 'Test Year' is 2019 and 'Test Type' is X which is science.

In [7]:
df_all.drop(columns=['Filler','Test Year','Test ID','Test Type'], inplace=True)

In [8]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986765 entries, 0 to 986764
Data columns (total 25 columns):
County Code                                                  986765 non-null int64
District Code                                                986765 non-null int64
School Code                                                  986765 non-null int64
Demographic ID                                               986765 non-null int64
Total Number Tested at Entity Level and Demographic          751470 non-null float64
Total Number Tested at this Demographic with Valid Scores    751470 non-null float64
Grade                                                        986765 non-null int64
CAST Reported Enrollment                                     693722 non-null float64
Total Number of Students Tested                              693722 non-null float64
Mean Scale Score                                             292201 non-null float64
Percentage Standard Exceeded                     

In [9]:
df_all[['Mean Scale Score','Percentage Standard Exceeded','Percentage Standard Met','Percentage Standard Met and Above','Percentage Standard Nearly Met','Percentage Standard Not Met']]

,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met
0,201.0,11.98,19.74,31.72,49.42,18.86
1,200.4,12.01,19.28,31.29,48.02,20.68
2,201.7,11.95,20.22,32.17,50.91,16.93
3,205.3,14.72,23.66,38.37,48.93,12.70
4,216.9,31.37,29.47,60.84,34.49,4.67
...,...,...,...,...,...,...
986760,NaN,NaN,NaN,NaN,NaN,NaN
986761,NaN,NaN,NaN,NaN,NaN,NaN
986762,NaN,NaN,NaN,NaN,NaN,NaN
986763,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_all.isnull().sum()

County Code                                                       0
District Code                                                     0
School Code                                                       0
Demographic ID                                                    0
Total Number Tested at Entity Level and Demographic          235295
Total Number Tested at this Demographic with Valid Scores    235295
Grade                                                             0
CAST Reported Enrollment                                     293043
Total Number of Students Tested                              293043
Mean Scale Score                                             694564
Percentage Standard Exceeded                                 470959
Percentage Standard Met                                      470959
Percentage Standard Met and Above                            470959
Percentage Standard Nearly Met                               470959
Percentage Standard Not Met                     

In [11]:
cols = list(df_all.columns.str.contains('Code|ID|Grade', regex=True))
df_all[df_all.columns[cols]] = df_all[df_all.columns[cols]].astype('category')

In [12]:
df_all.dtypes

County Code                                                  category
District Code                                                category
School Code                                                  category
Demographic ID                                               category
Total Number Tested at Entity Level and Demographic           float64
Total Number Tested at this Demographic with Valid Scores     float64
Grade                                                        category
CAST Reported Enrollment                                      float64
Total Number of Students Tested                               float64
Mean Scale Score                                              float64
Percentage Standard Exceeded                                  float64
Percentage Standard Met                                       float64
Percentage Standard Met and Above                             float64
Percentage Standard Nearly Met                                float64
Percentage Standard 

In [13]:
df_entities.shape

(11182, 10)

In [14]:
df_entities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11182 entries, 0 to 11181
Data columns (total 10 columns):
County Code      11182 non-null int64
District Code    11182 non-null int64
School Code      11182 non-null int64
Filler           0 non-null float64
Test Year        11182 non-null int64
Type ID          11182 non-null int64
County Name      11182 non-null object
District Name    11123 non-null object
School Name      10099 non-null object
Zip Code         11182 non-null object
dtypes: float64(1), int64(5), object(4)
memory usage: 873.7+ KB


In [15]:
df_entities.head()

,County Code,District Code,School Code,Filler,Test Year,Type ID,County Name,District Name,School Name,Zip Code
0,37,68056,114686,NaN,2019,7,San Diego,Del Mar Union Elementary,Ocean Air,92130
1,37,68056,6038111,NaN,2019,7,San Diego,Del Mar Union Elementary,Del Mar Heights Elementary,92014
2,37,68056,6088983,NaN,2019,7,San Diego,Del Mar Union Elementary,Del Mar Hills Elementary,92014
3,37,68056,6110696,NaN,2019,7,San Diego,Del Mar Union Elementary,Carmel Del Mar Elementary,92130
4,37,68056,6115620,NaN,2019,7,San Diego,Del Mar Union Elementary,Ashley Falls Elementary,92130


In [16]:
df_entities.drop(columns=['Filler','Test Year','Type ID'], inplace=True)

In [17]:
cat_cols = list(df_entities.columns.str.contains('Code|ID|Grade', regex=True))
df_entities[df_entities.columns[cat_cols]] = df_entities[df_entities.columns[cat_cols]].astype('category')
str_cols = list(df_entities.columns.str.contains('Name', regex=True))
df_entities[df_entities.columns[str_cols]] = df_entities[df_entities.columns[str_cols]].astype('str')


In [18]:
df_entities.dtypes

County Code      category
District Code    category
School Code      category
County Name        object
District Name      object
School Name        object
Zip Code         category
dtype: object

In [19]:
df_entities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11182 entries, 0 to 11181
Data columns (total 7 columns):
County Code      11182 non-null category
District Code    11182 non-null category
School Code      11182 non-null category
County Name      11182 non-null object
District Name    11182 non-null object
School Name      11182 non-null object
Zip Code         11182 non-null category
dtypes: category(4), object(3)
memory usage: 840.2+ KB


    Loading the subgroups data to understand the ID

In [20]:
df_subgroups  = pd.read_csv("../data/Subgroups.txt", parse_dates=True, na_values=na_values, index_col='Demographic ID')

In [21]:
df_subgroups.drop(columns=['Demographic ID Num'], inplace=True)
df_subgroups.head()

,Demographic Name,Student Group
Demographic ID,,
1,All Students,All Students
3,Male,Gender
4,Female,Gender
6,Fluent English proficient and English only,English-Language Fluency
7,Initial fluent English proficient (IFEP),English-Language Fluency


In [22]:
df_tests  = pd.read_csv("../data/Tests.txt", parse_dates=True, na_values=na_values, index_col='Test ID')

In [23]:
df_tests.drop(columns=['Test ID Num'], inplace=True)

In [24]:
df_tests.head()

,Test Name
Test ID,
1,SB - English Language Arts/Literacy
2,SB - Mathematics
3,CAA - English Language Arts/Literacy
4,CAA - Mathematics
17,CAST - California Science Test


Since we are considering just CAST (Science Test) this table is just for an info.

We will now use house prices from zillow to assign the median price of houses for a particular zipcode.

In [25]:
df_home_price = pd.read_csv("../data/Zillow_median_monthly_price.csv", na_values=na_values)

In [26]:
df_home_price.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,240227.0,...,1263498.0,1258462.0,1255108.0,1237464.0,1213945.0,1184708.0,1172697.0,1163787.0,1161592.0,1163169.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,209848.0,...,488603.0,487610.0,486842.0,486758.0,487956.0,489569.0,490612.0,490770.0,491112.0,492290.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,247618.0,...,1156671.0,1159676.0,1179068.0,1198812.0,1212277.0,1212462.0,1222241.0,1227790.0,1233483.0,1236976.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,200202.0,...,335844.0,335848.0,335952.0,335676.0,335851.0,336272.0,337283.0,338136.0,338624.0,338802.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,284312.0,...,639873.0,638457.0,637151.0,636880.0,637169.0,637791.0,637309.0,636881.0,636623.0,637448.0


In [27]:
df_home_price.State.dtype

dtype('O')

In [28]:
df_home_price = df_home_price[df_home_price.State.str.contains('CA')]

In [29]:
print(df_home_price.RegionType.value_counts()), df_home_price.shape[0]


Zip    1685
Name: RegionType, dtype: int64


(None, 1685)

In [30]:
df_home_price.drop(columns=['SizeRank','RegionName','RegionType','StateName','State','Metro','CountyName','City'],inplace=True)

In [31]:
df_home_price.head()

,RegionID,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,1996-07-31,1996-08-31,1996-09-30,...,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31
13,97564,330387.0,328880.0,328468.0,327403.0,327227.0,326972.0,327140.0,327802.0,329051.0,...,1183296.0,1191693.0,1203702.0,1207570.0,1211730.0,1210772.0,1212158.0,1207935.0,1202748.0,1200641.0
21,96107,170875.0,171206.0,171570.0,172226.0,172827.0,173253.0,173568.0,173630.0,173464.0,...,658057.0,660351.0,664511.0,668824.0,675012.0,680449.0,685447.0,687374.0,689194.0,694206.0
38,97771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,586467.0,586717.0,589478.0,588441.0,588778.0,590528.0,594184.0,594005.0,594274.0,595454.0
42,96027,178004.0,178386.0,178126.0,177806.0,177327.0,177019.0,177291.0,177304.0,177575.0,...,1432077.0,1440036.0,1451605.0,1460735.0,1475234.0,1487485.0,1499307.0,1506100.0,1512793.0,1522780.0
86,97711,184135.0,183166.0,182891.0,182165.0,181979.0,181339.0,180736.0,180342.0,180366.0,...,1050455.0,1053680.0,1061628.0,1073929.0,1082567.0,1085791.0,1083617.0,1078555.0,1074555.0,1071670.0


In [32]:
df_home_price.rename(columns={'RegionID':'Zip Code'}, inplace=True)

In [33]:
df_home_price.set_index('Zip Code', inplace=True)

In [34]:
df_home_price.columns = pd.to_datetime(df_home_price.columns)
df_2019_prices = df_home_price[[ y for y in df_home_price.columns if y.year == 2019]]

In [35]:
df_2019_prices

,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31
Zip Code,,,,,,,,,,,,
97564,1181988.0,1181523.0,1180127.0,1173613.0,1177612.0,1182537.0,1185327.0,1180963.0,1178105.0,1183296.0,1191693.0,1203702.0
96107,649093.0,646361.0,644176.0,644193.0,648523.0,653907.0,655255.0,654948.0,655326.0,658057.0,660351.0,664511.0
97771,591852.0,590413.0,588816.0,586887.0,588667.0,589770.0,589269.0,587532.0,585711.0,586467.0,586717.0,589478.0
96027,1353884.0,1353547.0,1354847.0,1361135.0,1376452.0,1392587.0,1403224.0,1411147.0,1420920.0,1432077.0,1440036.0,1451605.0
97711,1015768.0,1010685.0,1013889.0,1022144.0,1032184.0,1041480.0,1046741.0,1044814.0,1047159.0,1050455.0,1053680.0,1061628.0
...,...,...,...,...,...,...,...,...,...,...,...,...
97307,43614.0,43603.0,43595.0,43557.0,43448.0,43337.0,42991.0,42617.0,42212.0,42057.0,41931.0,41850.0
98530,197599.0,197341.0,197421.0,196128.0,195670.0,196711.0,200272.0,203531.0,205737.0,207472.0,209201.0,210799.0
98152,350419.0,353538.0,356945.0,358001.0,358245.0,357564.0,353687.0,349208.0,344924.0,342777.0,341630.0,340800.0


We find the median across all the months of 2019

In [36]:
df_2019_prices['Median'] = df_2019_prices.median(axis=1, skipna=None)

C:\Users\harid\.conda\envs\TestEnv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
df_2019_prices = df_2019_prices['Median']

In [38]:
df_2019_prices = df_2019_prices.reset_index()


In [39]:
df_2019_prices.head()

,Zip Code,Median
0,97564,1181755.5
1,96107,654427.5
2,97771,588741.5
3,96027,1397905.5
4,97711,1043147.0


In [40]:
df_2019_prices.dtypes

Zip Code      int64
Median      float64
dtype: object

Let's now go ahead and join tables based on common columns.

##### 1. Merging entities table with the grades table.

In [41]:
df_all.head()

,County Code,District Code,School Code,Demographic ID,Total Number Tested at Entity Level and Demographic,Total Number Tested at this Demographic with Valid Scores,Grade,CAST Reported Enrollment,Total Number of Students Tested,Mean Scale Score,...,Total Number of Students with Valid Scores,Life Sciences Domain Percent Below Standard,Life Sciences Domain Percent Near Standard,Life Sciences Domain Percent Above Standard,Physical Sciences Domain Percent Below Standard,Physical Sciences Domain Percent Near Standard,Physical Sciences Domain Percent Above Standard,Earth and Space Sciences Domain Percent Below Standard,Earth and Space Sciences Domain Percent Near Standard,Earth and Space Sciences Domain Percent Above Standard
0,0,0,0,1,1477006.0,1474994.0,5,463976.0,456604.0,201.0,...,456221.0,45.09,42.59,12.32,44.12,41.58,14.30,41.61,46.71,11.68
1,0,0,0,3,752970.0,751862.0,5,238673.0,234571.0,200.4,...,234372.0,45.64,41.96,12.40,45.70,39.76,14.53,43.15,45.53,11.33
2,0,0,0,4,724036.0,723132.0,5,225303.0,222033.0,201.7,...,221849.0,44.51,43.26,12.23,42.45,43.49,14.06,39.98,47.97,12.05
3,0,0,0,6,1285057.0,1283476.0,5,374237.0,368370.0,205.3,...,368072.0,36.81,48.15,15.04,35.78,46.74,17.48,33.50,52.22,14.28
4,0,0,0,7,67405.0,67369.0,5,18116.0,17986.0,216.9,...,17976.0,20.08,51.35,28.57,18.92,46.77,34.31,17.49,54.03,28.48


In [42]:
df_entities

,County Code,District Code,School Code,County Name,District Name,School Name,Zip Code
0,37,68056,114686,San Diego,Del Mar Union Elementary,Ocean Air,92130
1,37,68056,6038111,San Diego,Del Mar Union Elementary,Del Mar Heights Elementary,92014
2,37,68056,6088983,San Diego,Del Mar Union Elementary,Del Mar Hills Elementary,92014
3,37,68056,6110696,San Diego,Del Mar Union Elementary,Carmel Del Mar Elementary,92130
4,37,68056,6115620,San Diego,Del Mar Union Elementary,Ashley Falls Elementary,92130
...,...,...,...,...,...,...,...
11177,37,68049,138313,San Diego,University Prep,University Prep,91764
11178,37,68049,6038095,San Diego,Dehesa Elementary,Dehesa Elementary,92019
11179,37,68049,6119564,San Diego,Dehesa Charter,Dehesa Charter,92026
11180,37,68056,0,San Diego,Del Mar Union Elementary,nan,


In [43]:
df_all.columns

Index(['County Code', 'District Code', 'School Code', 'Demographic ID',
       'Total Number Tested at Entity Level and Demographic',
       'Total Number Tested at this Demographic with Valid Scores', 'Grade',
       'CAST Reported Enrollment', 'Total Number of Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Total Number of Students with Valid Scores',
       'Life Sciences Domain Percent Below Standard',
       'Life Sciences Domain Percent Near Standard',
       'Life Sciences Domain Percent Above Standard',
       'Physical Sciences Domain Percent Below Standard',
       'Physical Sciences Domain Percent Near Standard',
       'Physical Sciences Domain Percent Above Standard',
       'Earth and Space Sciences Domain Percent Below Standard',
       'Earth and Space Sciences Domain Percent Near Standard',
 

In [44]:
grade_entities = pd.merge(df_all, df_entities, how='inner', on=['County Code','District Code','School Code'])

In [45]:
grade_entities.head()

,County Code,District Code,School Code,Demographic ID,Total Number Tested at Entity Level and Demographic,Total Number Tested at this Demographic with Valid Scores,Grade,CAST Reported Enrollment,Total Number of Students Tested,Mean Scale Score,...,Physical Sciences Domain Percent Below Standard,Physical Sciences Domain Percent Near Standard,Physical Sciences Domain Percent Above Standard,Earth and Space Sciences Domain Percent Below Standard,Earth and Space Sciences Domain Percent Near Standard,Earth and Space Sciences Domain Percent Above Standard,County Name,District Name,School Name,Zip Code
0,0,0,0,1,1477006.0,1474994.0,5,463976.0,456604.0,201.0,...,44.12,41.58,14.30,41.61,46.71,11.68,State of California,nan,nan,
1,0,0,0,3,752970.0,751862.0,5,238673.0,234571.0,200.4,...,45.70,39.76,14.53,43.15,45.53,11.33,State of California,nan,nan,
2,0,0,0,4,724036.0,723132.0,5,225303.0,222033.0,201.7,...,42.45,43.49,14.06,39.98,47.97,12.05,State of California,nan,nan,
3,0,0,0,6,1285057.0,1283476.0,5,374237.0,368370.0,205.3,...,35.78,46.74,17.48,33.50,52.22,14.28,State of California,nan,nan,
4,0,0,0,7,67405.0,67369.0,5,18116.0,17986.0,216.9,...,18.92,46.77,34.31,17.49,54.03,28.48,State of California,nan,nan,


##### 2. Merging the above table with home price table

In [46]:
df_2019_prices['Zip Code'] = df_2019_prices['Zip Code'].astype('str')

In [47]:
df_2019_prices.dtypes

Zip Code     object
Median      float64
dtype: object

In [48]:
grade_entities['Zip Code'] = grade_entities['Zip Code'].astype('str')

In [49]:
grade_entities.shape

(986765, 29)

In [50]:
df_geh= pd.merge(grade_entities, df_2019_prices, how='inner', on=['Zip Code'])

##### 3. Merge with Subgroup ID Table

In [51]:
df_final = pd.merge(df_subgroups, df_geh, how='inner', on=['Demographic ID'])

In [52]:
df_final.head()

,Demographic ID,Demographic Name,Student Group,County Code,District Code,School Code,Total Number Tested at Entity Level and Demographic,Total Number Tested at this Demographic with Valid Scores,Grade,CAST Reported Enrollment,...,Physical Sciences Domain Percent Near Standard,Physical Sciences Domain Percent Above Standard,Earth and Space Sciences Domain Percent Below Standard,Earth and Space Sciences Domain Percent Near Standard,Earth and Space Sciences Domain Percent Above Standard,County Name,District Name,School Name,Zip Code,Median
0,1,All Students,All Students,2,61333,6002794,21.0,21.0,5,11.0,...,54.55,9.09,36.36,63.64,0.00,Alpine,Alpine County Unified,Diamond Valley Elementary,96120,3042249.5
1,1,All Students,All Students,2,61333,6002794,21.0,21.0,8,10.0,...,NaN,NaN,NaN,NaN,NaN,Alpine,Alpine County Unified,Diamond Valley Elementary,96120,3042249.5
2,1,All Students,All Students,2,61333,6002794,21.0,21.0,13,21.0,...,57.14,14.29,33.33,42.86,23.81,Alpine,Alpine County Unified,Diamond Valley Elementary,96120,3042249.5
3,1,All Students,All Students,6,10066,136580,NaN,NaN,5,NaN,...,NaN,NaN,NaN,NaN,NaN,Colusa,Colusa County Office of Education,Coastal Buttes,95987,692087.0
4,1,All Students,All Students,6,10066,136580,NaN,NaN,8,NaN,...,NaN,NaN,NaN,NaN,NaN,Colusa,Colusa County Office of Education,Coastal Buttes,95987,692087.0


### Imputing Missing values

We can ignore the following columns:
'Mean value score'is a value that determines the 'Percentage Standard', which is already known so we can omit it. 
'Total Number Tested/Valid Scores' can be ignored
Percentage Standard Met and Above  is sum of  Percentage Standard Exceeded & Percentage Standard Met


In [53]:
df_final.isnull().sum()

Demographic ID                                                   0
Demographic Name                                                 0
Student Group                                                    0
County Code                                                      0
District Code                                                    0
School Code                                                      0
Total Number Tested at Entity Level and Demographic           5179
Total Number Tested at this Demographic with Valid Scores     5179
Grade                                                            0
CAST Reported Enrollment                                      6852
Total Number of Students Tested                               6852
Mean Scale Score                                             12901
Percentage Standard Exceeded                                 10312
Percentage Standard Met                                      10312
Percentage Standard Met and Above                            1

In [54]:
df_final['School Name'].str.contains("Alternative" or "Program").any().sum()

1

In [55]:
df_final.drop(columns=['Total Number Tested at Entity Level and Demographic', 'Total Number Tested at this Demographic with Valid Scores', 'CAST Reported Enrollment', 'Total Number of Students Tested','Mean Scale Score', 'Percentage Standard Met and Above'], inplace=True)

In [56]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16035 entries, 0 to 16034
Data columns (total 26 columns):
Demographic ID                                            16035 non-null int64
Demographic Name                                          16035 non-null object
Student Group                                             16035 non-null object
County Code                                               16035 non-null category
District Code                                             16035 non-null category
School Code                                               16035 non-null category
Grade                                                     16035 non-null category
Percentage Standard Exceeded                              5723 non-null float64
Percentage Standard Met                                   5723 non-null float64
Percentage Standard Nearly Met                            5723 non-null float64
Percentage Standard Not Met                               5723 non-null float64
Tota

In [57]:
df_final[df_final.isnull().any(axis=1)].head()

,Demographic ID,Demographic Name,Student Group,County Code,District Code,School Code,Grade,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Nearly Met,...,Physical Sciences Domain Percent Near Standard,Physical Sciences Domain Percent Above Standard,Earth and Space Sciences Domain Percent Below Standard,Earth and Space Sciences Domain Percent Near Standard,Earth and Space Sciences Domain Percent Above Standard,County Name,District Name,School Name,Zip Code,Median
1,1,All Students,All Students,2,61333,6002794,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Alpine,Alpine County Unified,Diamond Valley Elementary,96120,3042249.5
3,1,All Students,All Students,6,10066,136580,5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Colusa,Colusa County Office of Education,Coastal Buttes,95987,692087.0
4,1,All Students,All Students,6,10066,136580,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Colusa,Colusa County Office of Education,Coastal Buttes,95987,692087.0
5,1,All Students,All Students,6,10066,136580,13,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Colusa,Colusa County Office of Education,Coastal Buttes,95987,692087.0
6,1,All Students,All Students,6,10066,630111,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Colusa,Colusa County Office of Education,S. William Abel Community,95987,692087.0


In [58]:
df = df_final.dropna(axis=0)

In [62]:
df.columns

Index(['Demographic ID', 'Demographic Name', 'Student Group', 'County Code',
       'District Code', 'School Code', 'Grade', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Nearly Met',
       'Percentage Standard Not Met',
       'Total Number of Students with Valid Scores',
       'Life Sciences Domain Percent Below Standard',
       'Life Sciences Domain Percent Near Standard',
       'Life Sciences Domain Percent Above Standard',
       'Physical Sciences Domain Percent Below Standard',
       'Physical Sciences Domain Percent Near Standard',
       'Physical Sciences Domain Percent Above Standard',
       'Earth and Space Sciences Domain Percent Below Standard',
       'Earth and Space Sciences Domain Percent Near Standard',
       'Earth and Space Sciences Domain Percent Above Standard', 'County Name',
       'District Name', 'School Name', 'Zip Code', 'Median'],
      dtype='object')

We need to eliminate entries that are not schools. For eg anything with 'Program' or 'Alternative' in their title can be eliminated.

In [63]:
df = df[~df["School Name"].str.contains('Program|Alternative', case=False, na=False)]

We will save the final dataframe into a csv for futher processing.

In [64]:
df.to_csv('cleaned_school_df.csv', sep='\t')